In [1]:
import pandas as pd
from sqlalchemy import create_engine
pd.options.mode.chained_assignment = None

In [2]:
csv_file = "Resources/elonmusk.csv"
elon_musk_df = pd.read_csv(csv_file)
elon_musk_df.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1.282940e+18,1.282930e+18,1.594710e+12,7/14/20,7:28:03,UTC,44196397,elonmusk,Elon Musk,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '44196397', 'username': 'elonmusk...",NaN,NaN,NaN,NaN
1,1.282840e+18,1.282800e+18,1.594690e+12,7/14/20,1:10:26,UTC,44196397,elonmusk,Elon Musk,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '44196397', 'username': 'elonmusk...",NaN,NaN,NaN,NaN
2,1.282810e+18,1.282760e+18,1.594680e+12,7/13/20,22:34:13,UTC,44196397,elonmusk,Elon Musk,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '44196397', 'username': 'elonmusk...",NaN,NaN,NaN,NaN
3,1.282800e+18,1.282670e+18,1.594680e+12,7/13/20,22:12:52,UTC,44196397,elonmusk,Elon Musk,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '44196397', 'username': 'elonmusk...",NaN,NaN,NaN,NaN
4,1.282800e+18,1.282740e+18,1.594680e+12,7/13/20,22:12:26,UTC,44196397,elonmusk,Elon Musk,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '44196397', 'username': 'elonmusk...",NaN,NaN,NaN,NaN


In [3]:
elon_musk_tweet_df = elon_musk_df[['date', 'tweet']]
elon_musk_tweet_df.head()

,date,tweet
0,7/14/20,Cute
1,7/14/20,Wow
2,7/13/20,Reusability is essential. A rocket that is sin...
3,7/13/20,Wild times!
4,7/13/20,We’re being extra paranoid. Maximizing probabi...


In [4]:
csv_file_2 = "Resources/datasets_756428_1355563_tesla-stocks-data.csv"
tesla_df = pd.read_csv(csv_file_2)
tesla_df.head()

,Date,Close/Last,Volume,Open,High,Low
0,7/17/20,"$1,500.84",9329972,"$1,513.45","$1,537.51","$1,490"
1,7/16/20,"$1,500.64",14300790,"$1,477.16","$1,531.71","$1,466"
2,7/15/20,"$1,546.01",16367830,"$1,543","$1,550","$1,457"
3,7/14/20,"$1,516.80",23418140,"$1,556","$1,590","$1,431"
4,7/13/20,"$1,497.06",38985360,"$1,659","$1,794.99","$1,471.11"


In [5]:
tesla_df.columns

Index(['Date', ' Close/Last', ' Volume', ' Open', ' High', ' Low'], dtype='object')

In [6]:
tesla_df.reset_index(drop=True)

,Date,Close/Last,Volume,Open,High,Low
0,7/17/20,"$1,500.84",9329972,"$1,513.45","$1,537.51","$1,490"
1,7/16/20,"$1,500.64",14300790,"$1,477.16","$1,531.71","$1,466"
2,7/15/20,"$1,546.01",16367830,"$1,543","$1,550","$1,457"
3,7/14/20,"$1,516.80",23418140,"$1,556","$1,590","$1,431"
4,7/13/20,"$1,497.06",38985360,"$1,659","$1,794.99","$1,471.11"
...,...,...,...,...,...,...
2512,7/26/10,$20.95,922178,$21.50,$21.50,$20.30
2513,7/23/10,$21.29,653549,$21.19,$21.56,$21.06
2514,7/22/10,$21,957271,$20.50,$21.25,$20.37
2515,7/21/10,$20.22,1251492,$20.66,$20.90,$19.50


In [7]:
tesla_stock_df = tesla_df[['Date', ' Close/Last', ' Open', ' High', ' Low']]
tesla_stock_df.head()

,Date,Close/Last,Open,High,Low
0,7/17/20,"$1,500.84","$1,513.45","$1,537.51","$1,490"
1,7/16/20,"$1,500.64","$1,477.16","$1,531.71","$1,466"
2,7/15/20,"$1,546.01","$1,543","$1,550","$1,457"
3,7/14/20,"$1,516.80","$1,556","$1,590","$1,431"
4,7/13/20,"$1,497.06","$1,659","$1,794.99","$1,471.11"


In [8]:
tesla_stock_df.rename(columns={'Date':'date', ' Close/Last':'Close', ' Open':'Open', ' High':'High', ' Low':'Low'},inplace=True)
tesla_stock_df.head()

,date,Close,Open,High,Low
0,7/17/20,"$1,500.84","$1,513.45","$1,537.51","$1,490"
1,7/16/20,"$1,500.64","$1,477.16","$1,531.71","$1,466"
2,7/15/20,"$1,546.01","$1,543","$1,550","$1,457"
3,7/14/20,"$1,516.80","$1,556","$1,590","$1,431"
4,7/13/20,"$1,497.06","$1,659","$1,794.99","$1,471.11"


In [9]:
tesla_stock_df['Close']=tesla_stock_df['Close'].str.lstrip('$').str.rstrip(' ').replace(',','')
tesla_stock_df=tesla_stock_df.replace(',','', regex=True)
tesla_stock_df['Close']=tesla_stock_df['Close'].astype(float)

In [10]:
tesla_stock_df['Open']=tesla_stock_df['Open'].str.lstrip('$').str.rstrip(' ').replace(',','')
tesla_stock_df=tesla_stock_df.replace(',','', regex=True)
tesla_stock_df['Open']=tesla_stock_df['Open'].astype(float)

In [11]:
tesla_stock_df['High']=tesla_stock_df['High'].str.lstrip('$').str.rstrip(' ').replace(',','')
tesla_stock_df=tesla_stock_df.replace(',','', regex=True)
tesla_stock_df['High']=tesla_stock_df['High'].astype(float)

In [12]:
tesla_stock_df['Low']=tesla_stock_df['Low'].str.lstrip('$').str.rstrip(' ')
tesla_stock_df=tesla_stock_df.replace(',','', regex=True)
tesla_stock_df['Low']=tesla_stock_df['Low'].astype(float)

In [13]:
tesla_stock_df.dtypes

date      object
Close    float64
Open     float64
High     float64
Low      float64
dtype: object

In [14]:
tesla_stock_df['Open_Close']=tesla_stock_df['Open']-tesla_stock_df['Close']
tesla_stock_df.head()

,date,Close,Open,High,Low,Open_Close
0,7/17/20,1500.84,1513.45,1537.51,1490.00,12.61
1,7/16/20,1500.64,1477.16,1531.71,1466.00,-23.48
2,7/15/20,1546.01,1543.00,1550.00,1457.00,-3.01
3,7/14/20,1516.80,1556.00,1590.00,1431.00,39.20
4,7/13/20,1497.06,1659.00,1794.99,1471.11,161.94


In [15]:
tesla_stock_df['High_Low']=tesla_stock_df['High']-tesla_stock_df['Low']
tesla_stock_df.head()

,date,Close,Open,High,Low,Open_Close,High_Low
0,7/17/20,1500.84,1513.45,1537.51,1490.00,12.61,47.51
1,7/16/20,1500.64,1477.16,1531.71,1466.00,-23.48,65.71
2,7/15/20,1546.01,1543.00,1550.00,1457.00,-3.01,93.00
3,7/14/20,1516.80,1556.00,1590.00,1431.00,39.20,159.00
4,7/13/20,1497.06,1659.00,1794.99,1471.11,161.94,323.88


In [16]:
elon_musk_tweet_df.head()

,date,tweet
0,7/14/20,Cute
1,7/14/20,Wow
2,7/13/20,Reusability is essential. A rocket that is sin...
3,7/13/20,Wild times!
4,7/13/20,We’re being extra paranoid. Maximizing probabi...


In [17]:
elon_tweet_df=elon_musk_tweet_df[elon_musk_tweet_df['tweet'].str.contains("tesla")]
elon_tweet_df

,date,tweet
198,7/1/20,Tesla Impact Report (repost). We do everything...
278,6/21/20,Tesla Bioweapon Defense Mode https://www.tesla...
347,6/15/20,Tesla Model S now first ever electric vehicle ...
392,6/10/20,"Posting Tesla Impact Report, this time with su..."
396,6/10/20,Wort reading the whole thing\n https://www.tes...
...,...,...
9201,5/1/15,"For more info on Tesla Energy, check out press..."
9225,4/11/15,"Just announced new Tesla 70D all-wheel drive, ..."
9231,4/4/15,We just got banned in West Virginia. Oh no. ht...
9232,4/1/15,Announcing the amazing Tesla Model W http://ww...


In [18]:
joined_data_df=pd.merge(elon_tweet_df,tesla_stock_df,how='inner',on='date')
joined_data_df

,date,tweet,Close,Open,High,Low,Open_Close,High_Low
0,7/1/20,Tesla Impact Report (repost). We do everything...,1119.63,1083.00,1135.33,1080.50,-36.63,54.83
1,6/15/20,Tesla Model S now first ever electric vehicle ...,990.90,917.79,998.84,908.50,-73.11,90.34
2,6/10/20,"Posting Tesla Impact Report, this time with su...",1025.05,991.88,1027.48,982.50,-33.17,44.98
3,6/10/20,Wort reading the whole thing\n https://www.tes...,1025.05,991.88,1027.48,982.50,-33.17,44.98
4,5/18/20,Tesla Full Self-Driving option cost rises by ~...,813.63,827.78,834.72,803.88,14.15,30.84
...,...,...,...,...,...,...,...,...
111,5/19/15,"This is not the full Gigafactory, it is just t...",247.14,248.43,251.00,246.15,1.29,4.85
112,5/4/15,Very funny post of a Tesla owner test driving ...,230.51,228.18,234.73,227.11,-2.33,7.62
113,5/1/15,"For more info on Tesla Energy, check out press...",226.03,229.94,231.77,220.41,3.91,11.36
114,4/1/15,Announcing the amazing Tesla Model W http://ww...,187.59,188.70,192.30,186.05,1.11,6.25
